The work is inspired by [Memo on Fresnel equations](https://seblagarde.wordpress.com/2013/04/29/memo-on-fresnel-equations/), I strongly suggest you check it out if not.

#### Fresnel Equation basis

The calculations of the reflectance $R$ (What we are looking for when we want to calculate the percentage of reflection and transmission) depend on p- and s-polarization of the incident ray. $R_s$ and $R_p$ are the reflectivity for the two planes of polarization. $R_s$ is perpendicular (s = German senkrecht) and $R_p$ is parallel. The reflectance $R$ for unpolarized light is the average of $R_s$ and $R_p$:

\begin{equation}
R = \frac {R_s + R_p} 2 = \frac {{r_\perp}{^2} + {r_\parallel}{^2}} 2
\end{equation}

Snell's law dielectric-dielectric interface :

\begin{align}
\theta_i &= \theta_o \\
\eta_i \sin \theta_i &= \eta_t \sin \theta_t
\end{align}

Fresnel equation dielectric-dielectric interface:

\begin{align}
& r_\perp = \frac {\eta_i {\cos \theta_i} - \eta_t {\cos \theta_t}} 
                  {\eta_i {\cos \theta_i} + \eta_t {\cos \theta_t}} 
\\
& r_\parallel = \frac {\eta_t {\cos \theta_i} - \eta_i {\cos \theta_t}} 
                      {\eta_t {\cos \theta_i} + \eta_i {\cos \theta_t}}
\end{align}


Conductor is silimar with dielectric except that it has complex IOR

\begin{equation}
\bar\eta_t = \eta_t + i\kappa_t
\end{equation}

Snell's law dielectric-conductor interface :
\begin{align}
\theta_i &= \theta_o \\
\eta_i \sin \theta_i &= (\eta_t + i\kappa_t) \sin \theta_t
\end{align}

Fresnel equation dielectric-conductor interface:

\begin{align}
& R_s = \frac {a^2 + b^2 - 2a\cos\theta + \cos{^2}\theta} 
                  {a^2 + b^2 + 2a\cos\theta + \cos{^2}\theta}
\\
& R_p = {r_\perp} \frac {\cos{^2}\theta \space {(a^2 + b^2)} - {2a} \space {\cos\theta} \space {\sin{^2}\theta + \sin{^4}\theta}}
                                {\cos{^2}\theta \space {(a^2 + b^2)} + {2a} \space {\cos\theta} \space {\sin{^2}\theta + \sin{^4}\theta}}
\\                    
& a^2 = \frac 1 {2{\eta_i}{^2}} ({\sqrt {(\eta_t^2 - \kappa_t^2 - \eta_i^2 \sin^2\theta)^2 + 4\eta_t^2\kappa_t^2}} + \eta_t^2 - \kappa_t^2 - \eta_i^2\sin^2\theta)
\\                    
& b^2 = \frac 1 {2{\eta_i}{^2}} ({\sqrt {(\eta_t^2 - \kappa_t^2 - \eta_i^2 \sin^2\theta)^2 + 4\eta_t^2\kappa_t^2}} - \eta_t^2 + \kappa_t^2 + \eta_i^2\sin^2\theta)
\end{align}

#### Fresnel Schlick’s approximation

There is two possible versions of the approximation.

The first Schlick’s approximation use $R(0)$ , the  Fresnel reflectance at 0° (reflectance at normal incidence), also name specular color in PBR game :

\begin{equation}
R = R(0) + (1 - R(0))(1-\cos\theta)^5
\end{equation}

For conductor:
\begin{equation}
R(0) = \frac {(\eta_t - \eta_i)^2 + \kappa_t^2} 
             {(\eta_t + \eta_i)^2 + \kappa_t^2} 
     = \mathbb{Re} {\frac {((\eta_t + i\kappa_t) - \eta_i)((\eta_t - i\kappa_t) - \eta_i)} 
                          {((\eta_t + i\kappa_t) + \eta_i)((\eta_t - i\kappa_t) + \eta_i)}}
\end{equation}

For dielectric:
\begin{equation}
R(0) = \frac {(\eta_t - \eta_i)^2} 
             {(\eta_t + \eta_i)^2} 
\end{equation}

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

air_ior = 1.000293
water_ior = 1.33

def fresnel(costheta_i, eta_i, eta_t, kappa_t):
    cosThetaI = costheta_i
    eta = eta_t / eta_i
    k = kappa_t / eta_i

    cosThetaI2 = cosThetaI*cosThetaI
    sinThetaI2 = 1-cosThetaI2
    sinThetaI4 = sinThetaI2*sinThetaI2

    temp1 = eta*eta - k*k - sinThetaI2
    a2pb2 = np.sqrt(temp1*temp1 + 4*k*k*eta*eta)
    a     = np.sqrt(0.5 * (a2pb2 + temp1))

    term1 = a2pb2 + cosThetaI2
    term2 = 2*a*cosThetaI

    Rs2 = (term1 - term2) / (term1 + term2)

    term3 = a2pb2*cosThetaI2 + sinThetaI4
    term4 = term2*sinThetaI2

    Rp2 = Rs2 * (term3 - term4) / (term3 + term4)

    return 0.5 * (Rp2 + Rs2)

def schlick(costheta_i, eta_i, eta_t, kappa_t):
    spec_color = ((eta_t - eta_i)**2 + kappa_t**2) / ((eta_t + eta_i)**2 + kappa_t**2)
    R = spec_color + (1.0 - spec_color) * (1.0 - costheta_i)**5
    return R

def comp_fresnel_impl(eta_t, kappa_t):
    xs = np.deg2rad(np.arange(0, 90, 1))
    ys = fresnel(np.cos(xs), air_ior, eta_t, kappa_t)
    ys_approx = schlick(np.cos(xs), air_ior, eta_t, kappa_t)

    plt.ylim(0.0, 1.0)
    plt.plot(xs, ys, label="Fresnel")
    plt.plot(xs, ys_approx, label="Schlick Approximation")
    plt.legend(loc='upper left')
    plt.show()

widgets.interact(comp_fresnel_impl, eta_t=widgets.FloatSlider(min=air_ior,max=20,step=0.01,value=water_ior), kappa_t=widgets.FloatSlider(min=0,max=10,step=0.01,value=0.0))
